## Captação dos dados VIA API

Utilizando NewsAPI para fazer resgate das notícias dos últimos 30 dias

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Obter a data atual
data_atual = datetime.now()
data = data_atual - timedelta(days=60)
data_formatada = data.strftime("%Y-%m-%d")

# URL da API
url = (f'https://newsapi.org/v2/everything?'
       'q=agronegócio&'
       'from={data_formatada}'
       'sortBy=popularity&'
       'apiKey=df22a809929b459796eda53179209ec0')

# Fazendo a requisição; making the requisition
response = requests.get(url)

if response.status_code == 200:
    noticias = response.json()
    articles = noticias.get("articles", [])
    
    dados = []

    if articles:
        for artigo in articles:
            dados.append({
                "Título": artigo.get("title", "N/A"),
                "Descrição": artigo.get("description", "N/A"),
                "Data de Publicação": artigo.get("publishedAt", "N/A"),
                "URL": artigo.get("url", "N/A"),
                "Fonte": artigo.get("source", {}).get("name", "N/A")  # Fonte do artigo; article font
            })
        
        df = pd.DataFrame(dados)
        display(df.head(10))
    else:
        print("Nenhuma notícia encontrada.")
else:
    print(f"Erro: {response.status_code}, {response.json().get('message')}")


,Título,Descrição,Data de Publicação,URL,Fonte
0,Como a tecnologia está gerando novas oportunid...,"Inovações como a Internet das Coisas, redes 5G...",2025-02-07T19:01:29Z,https://olhardigital.com.br/2025/02/07/agroneg...,Olhardigital.com.br
1,Eneva mira nova planta de gás natural em peque...,Empresa planeja fornecer combustível à logísti...,2025-02-05T22:46:01Z,https://www.terra.com.br/economia/eneva-mira-n...,Terra.com.br
2,Agronegócio é parte da solução para mudanças c...,"O Brasil, com sua matriz energética limpa e pr...",2025-02-17T14:50:26Z,https://www.terra.com.br/noticias/agronegocio-...,Terra.com.br
3,Amado Batista tem patrimônio avaliado em R$ 1 ...,"Segundo informações do jornal 'Extra', a carre...",2025-02-08T22:36:22Z,https://www.purepeople.com.br/noticia/amado-b...,Purepeople.com.br
4,Governo discute controle de preço com taxação ...,"Ministro da Agricultura avisou que se demite, ...",2025-02-26T11:00:35Z,https://veja.abril.com.br/coluna/jose-casado/g...,Abril.com.br
5,Amado Batista fatura alto com agro e soma patr...,"Apesar de ser um cantor famoso, Amado Batista ...",2025-02-10T18:45:03Z,https://www.terra.com.br/diversao/gente/amado-...,Terra.com.br
6,Mais da metade dos CEOs do agro veem negócios ...,"Pesquisa da PwC, com mais de 4,7 mil executivo...",2025-02-14T14:27:11Z,https://www.terra.com.br/economia/mais-da-meta...,Terra.com.br
7,Jeferson Miola: O que Nardes sabe sobre o dinh...,"Alô, PGR! Alô, PF! \nO post Jeferson Miola: O ...",2025-02-22T01:27:26Z,https://www.viomundo.com.br/politica/jeferson-...,Viomundo.com.br
8,JBS (JBSS3) e SLC (SLCE3) aguentam o estresse ...,O Itaú BBA divulgou na noite de terça-feira (2...,2025-01-29T12:20:36Z,https://dinheirama.com/jbs-jbss3-e-slc-slce3-a...,Dinheirama.com
9,Butão abre mercado para carne bovina congelada...,"O governo brasileiro informou a aceitação, pel...",2025-02-19T19:21:06Z,https://dinheirama.com/butao-abre-mercado-para...,Dinheirama.com


## ETL

In [2]:
# Creating a dataset copy so we don't modify the original one
df_copy = df.copy()

In [3]:
df_copy.tail()

,Título,Descrição,Data de Publicação,URL,Fonte
95,"CDBs lideram captação bancária em 2024, e esto...","Expansão reflete o bom momento da renda fixa, ...",2025-01-30T19:40:31Z,https://www.infomoney.com.br/onde-investir/cdb...,InfoMoney
96,Ranking Outliers InfoMoney: veja finalistas em...,A seleção tomou como base rankings quantitativ...,2025-02-04T08:00:00Z,https://www.infomoney.com.br/onde-investir/ran...,InfoMoney
97,Aniversário de São Paulo: papel da cidade na f...,Como descrever a maior cidade da América Latin...,2025-01-28T01:37:01Z,https://www.pragmatismopolitico.com.br/2025/01...,Pragmatismopolitico.com.br
98,Quem é o brasileiro preso por lavar R$ 2 bilhõ...,Acusado de ligações com o PCC e com a máfia it...,2025-02-14T11:33:57Z,https://veja.abril.com.br/coluna/maquiavel/que...,Abril.com.br
99,O uso responsável da água na agricultura,Conheça as técnicas agrícolas em atividade par...,2025-02-03T19:08:40Z,https://veja.abril.com.br/coluna/mundo-agro/o-...,Abril.com.br


In [4]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Título              93 non-null     object
 1   Descrição           100 non-null    object
 2   Data de Publicação  100 non-null    object
 3   URL                 100 non-null    object
 4   Fonte               100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [5]:
df_copy.isna().sum()

Título                7
Descrição             0
Data de Publicação    0
URL                   0
Fonte                 0
dtype: int64

In [6]:
# Exibir linhas com nulos, como os nuloes estão na coluna TITULO não precisaremos remove-los.
# Displaying rows with null data, as the nulls are in the TITLE column we will not need to remove them.
linhas_com_nulos = df_copy[df_copy.isnull().any(axis=1)]
linhas_com_nulos


,Título,Descrição,Data de Publicação,URL,Fonte
32,None,O Palmeiras está encaminhado com o seu terceir...,2025-02-14T17:22:01Z,https://www.uol.com.br/esporte/colunas/danilo-...,Uol.com.br
42,None,Ainda não se sabe qual será o tamanho e a magn...,2025-02-10T20:16:56Z,https://www.uol.com.br/ecoa/colunas/mariana-sg...,Uol.com.br
66,None,A PGR (Procuradoria-Geral da República) denunc...,2025-02-19T09:22:40Z,https://noticias.uol.com.br/politica/ultimas-n...,Uol.com.br
67,None,A guerra comercial entre Estados Unidos e Chin...,2025-02-04T19:11:50Z,https://economia.uol.com.br/noticias/redacao/2...,Uol.com.br
68,None,Após retornar à Casa Branca para mais um manda...,2025-02-26T08:56:06Z,https://economia.uol.com.br/mais/ultimas-notic...,Uol.com.br
85,None,A Globo comunicou oficialmente nesta terça-fei...,2025-01-28T22:02:36Z,https://www.uol.com.br/splash/noticias/2025/01...,Uol.com.br
86,None,Número de equipamentos cadastrados para pulver...,2025-02-09T23:05:37Z,https://noticias.uol.com.br/ultimas-noticias/d...,Uol.com.br


## Aqui vamos importar um modelo de IA para facilitar a classificação das notícias mesmo sem um ML próprio

dependencias:

In [7]:
# pip install -q -U google-generativeai

Adicionando a análise de sentimento utilizando o gemini.

In [ ]:
import google.generativeai as genai

# Configurar API Key
genai.configure(api_key= "API-Key")

# Criar modelo corretamente
modelo = genai.GenerativeModel("gemini-1.5-pro-latest")

# Função para analisar sentimento com Gemini
def analisar_sentimento(texto):
    try:
        prompt = f"Classifique a seguinte notícia como 'good', 'bad' ou 'neutral': {texto}"
        resposta = modelo.generate_content(prompt)
        return resposta.text.strip().lower() if resposta.text else "neutral"
    except Exception as e:
        print(f"Erro ao processar: {e}")
        return "neutral"

# Exemplo de uso
# Aplicar análise ao DataFrame
df["Sentimento"] = df["Descrição"].apply(lambda x: analisar_sentimento(x) if x != "N/A" else "neutral")

# Exibir DataFrame com sentimento
display(df.head(10))

Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro ao processar: 429 Resource has been exhausted (e.g. check quota).
Erro a

,Título,Descrição,Data de Publicação,URL,Fonte,Sentimento
0,Como a tecnologia está gerando novas oportunid...,"Inovações como a Internet das Coisas, redes 5G...",2025-02-07T19:01:29Z,https://olhardigital.com.br/2025/02/07/agroneg...,Olhardigital.com.br,a notícia é **good**. ela descreve como a tecn...
1,Eneva mira nova planta de gás natural em peque...,Empresa planeja fornecer combustível à logísti...,2025-02-05T22:46:01Z,https://www.terra.com.br/economia/eneva-mira-n...,Terra.com.br,"a notícia é **neutral**. a priori, o fornecim..."
2,Agronegócio é parte da solução para mudanças c...,"O Brasil, com sua matriz energética limpa e pr...",2025-02-17T14:50:26Z,https://www.terra.com.br/noticias/agronegocio-...,Terra.com.br,good.
3,Amado Batista tem patrimônio avaliado em R$ 1 ...,"Segundo informações do jornal 'Extra', a carre...",2025-02-08T22:36:22Z,https://www.purepeople.com.br/noticia/amado-b...,Purepeople.com.br,neutral. a notícia em si não expressa um julg...
4,Governo discute controle de preço com taxação ...,"Ministro da Agricultura avisou que se demite, ...",2025-02-26T11:00:35Z,https://veja.abril.com.br/coluna/jose-casado/g...,Abril.com.br,a notícia é **bad**. a possível demissão de um...
5,Amado Batista fatura alto com agro e soma patr...,"Apesar de ser um cantor famoso, Amado Batista ...",2025-02-10T18:45:03Z,https://www.terra.com.br/diversao/gente/amado-...,Terra.com.br,neutral.\n\na notícia simplesmente afirma fato...
6,Mais da metade dos CEOs do agro veem negócios ...,"Pesquisa da PwC, com mais de 4,7 mil executivo...",2025-02-14T14:27:11Z,https://www.terra.com.br/economia/mais-da-meta...,Terra.com.br,neutral. embora destaque uma crescente conscie...
7,Jeferson Miola: O que Nardes sabe sobre o dinh...,"Alô, PGR! Alô, PF! \nO post Jeferson Miola: O ...",2025-02-22T01:27:26Z,https://www.viomundo.com.br/politica/jeferson-...,Viomundo.com.br,neutral
8,JBS (JBSS3) e SLC (SLCE3) aguentam o estresse ...,O Itaú BBA divulgou na noite de terça-feira (2...,2025-01-29T12:20:36Z,https://dinheirama.com/jbs-jbss3-e-slc-slce3-a...,Dinheirama.com,neutral
9,Butão abre mercado para carne bovina congelada...,"O governo brasileiro informou a aceitação, pel...",2025-02-19T19:21:06Z,https://dinheirama.com/butao-abre-mercado-para...,Dinheirama.com,neutral


In [ ]:
# Configurar API Key
genai.configure(api_key= "API-Key")

# Listar os modelos disponíveis
modelos_disponiveis = genai.list_models()
for modelo in modelos_disponiveis:
    print(modelo.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experim